In [1]:
import os
import random
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

In [2]:
letter

(612.0, 792.0)

In [215]:
def get_random_elements(root, n, seed=None):

    random_elements_per_category, categories_list = [],[]

    if seed == None:
        seed = "".join([random.choice(['1','2','3','4','5','6','7','8','9','0']) for _ in range(6)])
        random.seed(seed)
    else:
        random.seed(seed)

    categories_folders = [os.path.join(root, d) for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]

    for category_folder in categories_folders:
        category = os.path.basename(category_folder)
        category_elements = [os.path.join(category_folder, file) for file in os.listdir(category_folder) if os.path.isfile(os.path.join(category_folder, file))]
        category_elements_images = [img for img in category_elements if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

        categories_list += [category]
        random_elements_per_category += [random.sample(category_elements_images, n)]
    
    return random_elements_per_category, categories_list, seed

In [216]:
def get_resize_data(image_list):
    MAX_WIDTH = 612 - (30*2)
    MAX_HEIGHT = 500
    new_size_images_list = []

    for img in image_list:
        image = Image.open(img)
        if image.height > MAX_HEIGHT or image.width > MAX_WIDTH:
            ratio = min(MAX_WIDTH / image.width, MAX_HEIGHT / image.height)
            new_height = int(image.height * ratio)
        else:
            new_height = image.height

        new_size_images_list.append(new_height)
    
    return new_size_images_list

In [217]:
def get_resize_image(img):
    MAX_WIDTH = 612 - (30*2)
    MAX_HEIGHT = 500
    image = Image.open(img)
    #image = ImageOps.exif_transpose(image) 
    if image.height > MAX_HEIGHT or image.width > MAX_WIDTH:
        ratio = min(MAX_WIDTH / image.width, MAX_HEIGHT / image.height)
        new_width = int(image.width * ratio)
        new_height = int(image.height * ratio)
    else:
        new_width, new_height = image.width, image.height
    return new_width, new_height

In [218]:
def optimize_layout(left_nodes, actual_page, actual_layout, final_layout_candidates):
    ITEMS_PER_PAGE = 3
    MARGIN_ITEMS = 25
    MAX_HEIGHT = 792 - 25 - MARGIN_ITEMS

    if len(left_nodes)==0:
        final_layout_candidates.append(actual_layout)
    
    else:
        #print('revisando el actual page', actual_page)
        #print('nodos faltantes', left_nodes)
        #size_left_nodes = [Image.open(img).height for img in left_nodes]
        size_left_nodes = get_resize_data(left_nodes)
        #size_actual_page = [Image.open(img).height for img in actual_page]
        size_actual_page = get_resize_data(actual_page)
        #print('size left nodes', size_left_nodes, 'size actual page', size_actual_page)

        if len(actual_layout) == 0:
            replace_flag = False
        else:
            actual_page = actual_layout[-1]
            replace_flag = True

        add_node_flag = False
        for item_index in range(len(left_nodes)):
            if sum(size_actual_page) + size_left_nodes[item_index] <= MAX_HEIGHT - (len(actual_page)*MARGIN_ITEMS) and len(actual_page)<=ITEMS_PER_PAGE-1:
                #print('se puede agregar',left_nodes[item_index], 'en', actual_page,'porque', size_left_nodes[item_index], 'cae en', size_actual_page )  
                actual_page += [left_nodes[item_index]]
                #print('actual list queda', actual_page)
                left_nodes.pop(left_nodes.index(left_nodes[item_index]))
                #print('left nodes queda', left_nodes)
                add_node_flag = True 
                break

        if add_node_flag == True:
            if replace_flag == True:
                actual_layout[-1] = actual_page
            else:
                actual_layout.append(actual_page)
        else:
            #print('no se agregó el ningun nodo')
            value = random.choice(left_nodes) 
            index = left_nodes.index(value)

            actual_layout.append([value])
            left_nodes.pop(index)
            actual_page = [value]
        
        #print('actual layout queda', actual_layout)
        #print()
            
        optimize_layout(left_nodes, actual_page, actual_layout, final_layout_candidates)

In [219]:
def get_final_layout(categories_list, seed):
    final = []
    random.seed(seed)
    for category in categories_list:

        layout_proposals_per_category = []

        for _ in range(10):
            #print('propuesta', _+1)
            for file in category:
                left_nodes = category.copy()
                item_index = left_nodes.index(file)
                left_nodes.pop(item_index)
                optimize_layout(left_nodes, [file], [], layout_proposals_per_category)
        
        min_layout = min(layout_proposals_per_category, key=len)
        #print(category)
        #print(min_layout)
        final += min_layout
        
    return final

In [220]:
def add_header(canvas_pdf, title, seed):
    PAGE_WIDTH = 612
    PAGE_HEIGHT = 792 - 15
    
    canvas_pdf.setFont("Helvetica-Bold", 16)
    canvas_pdf.drawCentredString(PAGE_WIDTH / 2, PAGE_HEIGHT-5, title.capitalize())

    canvas_pdf.setLineWidth(1)
    canvas_pdf.line(30, PAGE_HEIGHT - 10, PAGE_WIDTH - 30, PAGE_HEIGHT - 10) 

    canvas_pdf.setFont("Helvetica", 12)
    canvas_pdf.drawRightString(PAGE_WIDTH - 30, PAGE_HEIGHT -5, f"Forma: {seed}")

    canvas_pdf.setFont("Helvetica", 12)
    canvas_pdf.drawRightString(PAGE_WIDTH/3, PAGE_HEIGHT -5, f"ensayosaleatorios.streamlit.app")

In [221]:
def add_pdf_page(canvas_pdf, layout_page, categories_list, seed):
    X_CENTER = 30
    MARGIN = 25
    for item in categories_list:
        if item in layout_page[0]:
                title = item
                break
    add_header(canvas_pdf, title, seed)

    space_left = 792 - 25 - MARGIN
    for img_route in layout_page:
        new_width, new_height = get_resize_image(img_route)
        canvas_pdf.drawImage(img_route, X_CENTER, space_left - new_height - 5, new_width, new_height)
        canvas_pdf.setStrokeColorRGB(0, 0, 0)  # Color negro para el borde
        canvas_pdf.setLineWidth(1)
        #canvas_pdf.line(X_CENTER, space_left - new_height - 10, 612 - X_CENTER, space_left - new_height - 10) 
        #canvas_pdf.rect(X_CENTER - 5, space_left - new_height - 5, 612 - 50, new_height + 10)  # Borde con margen
        space_left -= new_height + MARGIN        

In [222]:
def create_pdf(layout_pages, categories_list, seed):
    PAGE_TYPE = letter
    output_file = 'resultado_final.pdf'

    canvas_pdf = canvas.Canvas(output_file, pagesize=PAGE_TYPE)
    for layout_page in layout_pages:
        add_pdf_page(canvas_pdf, layout_page, categories_list, seed)
        canvas_pdf.showPage() 
        
    canvas_pdf.save()

In [223]:
random_elements_per_category, categories_list, seed = get_random_elements('./matematicas',3)
layout_pages = get_final_layout(random_elements_per_category, seed)
create_pdf(layout_pages, categories_list, seed)